### Assignment Week 6

In [225]:
import pandas as pd
from pandas.compat import StringIO

#import data from dat file using StringIO library
temp=u"""
 1 1 1 1 1 1 0 1 1 0 0 0 0 0
 1 1 1 0 1 1 1 1 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 0 0 0 0 0
 1 0 1 1 1 1 1 1 0 0 0 0 0 0
 0 0 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 1 0 1 1 0 1 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 1 0 1 1 0 0 0 0 0
 0 0 0 0 1 0 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 0 0 1 0 0
 0 0 0 0 0 0 0 1 1 1 0 1 0 0
 0 0 0 0 0 0 0 1 1 1 0 1 1 1
 0 0 0 0 0 0 1 1 1 1 0 1 1 1
 0 0 0 0 0 1 1 0 1 1 1 1 1 1
 0 0 0 0 0 0 1 1 0 1 1 1 1 1
 0 0 0 0 0 0 0 1 1 1 0 1 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0"""

#after testing replace StringIO(temp) to filename
df = pd.read_csv(StringIO(temp), 
                 sep="\s+", 
                 names=['E1','E2','E3','E4','E5','E6','E7','E8','E9','E10','E11','E12','E13','E14'])

df.set_index(pd.Index(['EVELYN','LAURA','THERESA','BRENDA','CHARLOTTE','FRANCES','ELEANOR','PEARL','RUTH','VERNE','MYRNA','KATHERINE','SYLVIA','NORA','HELEN','DOROTHY','OLIVIA','FLORA']),inplace=True)

In [226]:
participants=[]

for event in list(df.columns):
    #finds out event's participant
    participant =list(df[df.loc[:,event]==1].index)
    #stores lists of participants for each event
    participants.append(friends)

In [227]:
# find all combinations of given length 
from itertools import combinations 
  
# and length 2 
friends_pairs_list=[]
for friends_set in friends_list: 
    friends_pairs = list(combinations(friends_set, 2))
    friends_pairs_list+=friends_pairs



In [228]:
#stores graph edges
friends = pd.DataFrame(friends_pairs_list,columns=['friend1','friend2'])
friends.head()

,friend1,friend2
0,EVELYN,LAURA
1,EVELYN,BRENDA
2,LAURA,BRENDA
3,EVELYN,LAURA
4,EVELYN,THERESA


In [229]:
#calculate edges weights
weights = friends.groupby(['friend1','friend2']).size().reset_index()
weights.columns = ['friend1','friend2','weight']

weights.sort_values('weight', ascending=False).head(10)


,friend1,friend2,weight
45,EVELYN,THERESA,7
38,EVELYN,LAURA,6
116,THERESA,BRENDA,6
114,SYLVIA,NORA,6
113,SYLVIA,HELEN,6
89,NORA,HELEN,6
65,KATHERINE,SYLVIA,6
30,EVELYN,BRENDA,6
66,LAURA,BRENDA,6
78,LAURA,THERESA,6


The table above shows women that most probaly became friens since they met with each other more friquently than with the other women.